In [ ]:
import os
import pandas as pd
import re
import tensorflow as tf
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib import colors
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.activations import linear, relu, sigmoid, softmax
from tensorflow.keras.optimizers import*
from sklearn.metrics import accuracy_score, recall_score, confusion_matrix, ConfusionMatrixDisplay
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.regularizers import l1, l2
from tensorflow.keras.layers import Dense, BatchNormalization

In [ ]:
pip show tensorflow

Name: tensorflow
Version: 2.15.0
Summary: TensorFlow is an open source machine learning framework for everyone.
Home-page: https://www.tensorflow.org/
Author: Google Inc.
Author-email: packages@tensorflow.org
License: Apache 2.0
Location: /usr/local/lib/python3.10/dist-packages
Requires: absl-py, astunparse, flatbuffers, gast, google-pasta, grpcio, h5py, keras, libclang, ml-dtypes, numpy, opt-einsum, packaging, protobuf, setuptools, six, tensorboard, tensorflow-estimator, tensorflow-io-gcs-filesystem, termcolor, typing-extensions, wrapt
Required-by: dopamine-rl, tf_keras


In [8]:
df = pd.read_csv(r"/users/eleves-a/2022/noam-joud-harold.ngoupeyou/dataset_flop_final.csv", header = 0)
df

FileNotFoundError: [Errno 2] No such file or directory: '/users/eleves-a/2022/noam-joud-harold.ngoupeyou/dataset_flop_final.csv'

In [ ]:
df["card_1_player"]=0
df["card_2_player"]=0
df["color_1_player"]=0
df["color_2_player"] =0
df["card_1_board"]= 0
df["card_2_board"] = 0
df["card_3_board"]= 0
df["color_3_board"]=0
df["color_2_board"] = 0
df["color_1_board"] =0


#df = df.drop(columns=['game_id', 'high_value', 'min_value', 'same_color'])
df['decision'] = df['decision'].replace({'checks': 'calls', 'bets': 'raises'})

df


In [ ]:
cards = df["cards"]
deck = df["flop"]
d = deck[6784]
j = 0
def get_value(s):
  if s == 'A':
    return 14
  if s=='K':
    return 13
  if s== 'Q':
    return 12
  if s== 'J':
    return 11
  if s == '1':
    return 10
  return int(s)

values_deck = []

if d[0] == "1":
  values_deck.append(10)
  j += 1
else :
  values_deck.append(get_value(d[0]))

j += 1
values_deck.append(d[j])
j +=2
if d[j] == "1":
  values_deck.append(10)
  j += 1
else :
  values_deck.append(get_value(d[j]))

j += 1
values_deck.append(d[j])

j +=2
if d[j] == "1":
  values_deck.append(10)
  j += 1
else :
  values_deck.append(get_value(d[j]))

j += 1
values_deck.append(d[j])


values_deck

In [ ]:
cards = df["cards"]
deck = df["flop"]
#cards

def get_value(s):
  if s == 'A':
    return 14
  if s=='K':
    return 13
  if s== 'Q':
    return 12
  if s== 'J':
    return 11
  if (s == '1') | (s=='T'):
    return 10
  return int(s)

def get_color(s):
  if s=='s':
    return 0
  elif s =='h':
    return 1
  elif s=='d':
    return 2
  else :
    return 3

for i in range(df.shape[0]):
  index = 0
  j = 0
  values = []
  values_deck = []
  c = df['cards'][i]
  d = df['flop'][i]
  if c[0] == "1":
    values.append(10)
    index+=1
  else :
    values.append(get_value(c[0]))

  index += 1
  values.append(c[index])
  index+=2
  if c[index] == "1":
    values.append(10)
    index+=1
  else :
    values.append(get_value(c[index]))

  index += 1
  values.append(c[index])
  if d[0] == "1":
    values_deck.append(10)
    j += 1
  else :
    values_deck.append(get_value(d[0]))

  j += 1
  values_deck.append(d[j])
  j +=2
  if d[j] == "1":
    values_deck.append(10)
    j += 1
  else :
    values_deck.append(get_value(d[j]))

  j += 1
  values_deck.append(d[j])

  j +=2
  if d[j] == "1":
    values_deck.append(10)
    j += 1
  else :
    values_deck.append(get_value(d[j]))

  j += 1
  values_deck.append(d[j])


  df["card_1_player"][i] = values[0]
  df["card_2_player"][i] = values[2]
  df["color_1_player"][i] = get_color(values[1])
  df["color_2_player"][i] = get_color(values[3])
  df["card_1_board"][i]= values_deck[0]
  df["card_2_board"][i] = values_deck[2]
  df["card_3_board"][i] = values_deck[4]
  df["color_3_board"][i] = get_color(values_deck[5])
  df["color_2_board"][i] = get_color(values_deck[3])
  df["color_1_board"][i] = get_color(values_deck[1])




  if df['decision'][i] == 'folds' :
    df['decision'][i] = 0
  elif df['decision'][i] == 'calls' :
    df['decision'][i] = 1
  elif df['decision'][i] == 'raises' :
    df['decision'][i] = 2
  if i == 0 :
    print(values_deck)


df


In [ ]:
n= df['decision'].nunique()
features = df.drop(columns=['game_id', 'cards', 'flop', 'mise en cas de raise/bet','hand_strength','hand_pos_potential','hand_neg_potential'])
M = features.corr()


plt.figure(figsize=(20, 16))
sns.heatmap(M, annot=True, cmap="coolwarm", linewidths=0.5, fmt=".2f", vmin=-1, vmax=1)
plt.title("Correlation Heatmap")
plt.show()

In [ ]:
pip install --upgrade tensorflow


In [ ]:
# Training
features = df.drop(columns=['game_id', 'decision', 'cards', 'flop',"mise en cas de raise/bet",'hand_strength','hand_pos_potential','hand_neg_potential']).values.astype(np.float32)
labels = df['decision']
labels = labels.values.astype(np.float32)
X_train, X_test, Y_train, Y_test = train_test_split(features, labels, random_state = 42)

model = Sequential(
    [
      tf.keras.Input(shape=(X_train.shape[1],)),
      Dense(units=64,activation='relu',name='layer1'),
      Dropout(0.2),
      Dense(units=32,activation='relu',name='layer2'),
      Dropout(0.2),
      Dense(units=32,activation='relu',name='layer3'),
      Dropout(0.2),
      Dense(units=3,activation='softmax',name='layer4')


    ], name = "flop_model1"
)

features = df.drop(columns=['game_id', 'decision', 'cards', 'flop',"mise en cas de raise/bet",'hand_strength','hand_pos_potential','hand_neg_potential']).values.astype(np.float32)
# features = df.drop(columns=['game_id', 'decision', 'cards', 'flop',"mise en cas de raise/bet",'hand_strength','hand_pos_potential','hand_neg_potential'])
labels = df['decision']
labels = labels.values.astype(np.float32)
X_train, X_test, Y_train, Y_test = train_test_split(features, labels, random_state = 42)

model.compile(
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    metrics = ['accuracy']
)

early_stopping = EarlyStopping(monitor='accuracy', patience=30, restore_best_weights=True)

history = model.fit(   X_train,Y_train,    epochs=1000,validation_data=(X_test, Y_test), callbacks=[early_stopping])


In [ ]:

y_true = Y_test
y_pred = model.predict(X_test)
y_pred = np.argmax(y_pred, axis=1)

ConfusionMatrixDisplay.from_predictions(y_true, y_pred)
plt.title('Matrice de Confusion')
plt.show()

f1 = f1_score(y_true, y_pred, average='macro') # 'binary' pour classification binaire, 'macro' pour multi-classes
print(f1)

In [ ]:
a = max(history.history['accuracy'])
b = max(history.history['val_accuracy'])
print(a)
print(b)

In [ ]:
model.save(r"/users/eleves-a/2022/mariem.ould-ismail/modele_flopv3.keras')


